In [1]:

from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense, LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import numpy as np

import random
import sys
import io
import re

C:\Users\shinj\anaconda3\envs\ML\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\shinj\anaconda3\envs\ML\lib\site-packages\numpy\.libs\libopenblas.noijjg62emaszi6nyurl6jbkm4evbgm7.gfortran-win_amd64.dll
C:\Users\shinj\anaconda3\envs\ML\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:\n%s" %


In [2]:

path = 'D:/shinj/Data/barkers.txt'
with io.open(path, encoding='utf-8') as f:
    text = f.read().lower()

text = re.sub(r'<.*>', '', text)
text = re.sub(r'\n', ' ', text)
text = re.sub(r' +', ' ', text)

print('corpus length:', len(text))

corpus length: 1469584


In [3]:
chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

total chars: 1527


In [4]:
maxlen = 40
step = 3
sentences = []
next_chars = []

for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))

print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)

for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

nb sequences: 489848
Vectorization...


In [5]:
print('Build model...')
model = Sequential()
model.add(LSTM(1024, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars), activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer=RMSprop(lr=0.001))

Build model...


In [6]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)


def on_epoch_end(epoch, _):
    print('\n----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(text) - maxlen - 1)
#     for diversity in [0.2, 0.5, 1.0, 1.2]:
#         print('----- diversity:', diversity)

    generated = ''
    sentence = text[start_index: start_index + maxlen]
    generated += sentence
    print('----- Generating with seed: "' + sentence + '"')
    sys.stdout.write(generated)

    for i in range(400):
        x_pred = np.zeros((1, maxlen, len(chars)))
        for t, char in enumerate(sentence):
            x_pred[0, t, char_indices[char]] = 1.

        preds = model.predict(x_pred, verbose=0)[0]
        next_index = sample(preds, 0.5)
        next_char = indices_char[next_index]

        generated += next_char
        sentence = sentence[1:] + next_char

        sys.stdout.write(next_char)
        sys.stdout.flush()
    print()

print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

In [ ]:
model.fit(x, y, batch_size=128, epochs=60, callbacks=[print_callback])


Epoch 1/60
3827/3827 [==============================] - ETA: 0s - loss: 3.1151
----- Generating text after Epoch: 0
----- Generating with seed: "." 그의 모습을 지켜 보고 있던 라수스는 머리 속에 기억나는 인물이 있"
." 그의 모습을 지켜 보고 있던 라수스는 머리 속에 기억나는 인물이 있었다. "그것을 알고 있는 것 같이 되었다. "그래 그것을 보내 아니지 그제 그런데 그 실수의 말에 있는 동연이 대답하고 있었다. 그래도 이렇게 해주 그것은 그 것 같아 모습니다. 그 아이 이상 되는 것 같습니다. 그것이 어떻게 되는 것 같습니다." " 음에 것 같습니다." "..............." "아. 이곳에 있는 것이라고 생각지도 않게 것 같습니다." "생각이 저 그렇게 이스반 왕국을 이해 자네에 대해 일어 있는 것 같습니다. 서연은 지금 자신을 이야기하는 것은 아니었다. "그렇게 그 어떻게 한 전에 있는 것 같군요." "어떻게 생각하는 것은 아리엘은 것이 아니라 많은 데 고개를 돌렸다. "그것이 아리엘 았다. 사람들은 움직이고 있는 것이 있는 것 같습니다." 미소를 지었다. "그것이 어떻게 하
3827/3827 [==============================] - 8734s 2s/step - loss: 3.1151
Epoch 2/60
3140/3827 [=======================>......] - ETA: 26:09 - loss: 2.5019